# SimpleSim Non-Holonomic Navigation Challenge

This notebook is made to train the best possible performance version of the final ReLOaD architecture, with:
- Variable budget
- Variable num targets
- Absolute information gain (entropy-based) reward
  - Not normalised against the number of targets or the length of the episode


## Install Dependencies and Stable Baselines3 Using Pip

In [1]:
# !pip install "stable-baselines3[extra]>=2.0.0a4"

### Setup Tensorboard Logging

In [2]:
import os

# # Clear any logs from previous runs
# !rm -rf ./logs/

# Load the TensorBoard notebook extension
%load_ext tensorboard

## Import Custom Gym Env

In [3]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces

# Import our main environment
from env_gym import SimpleSimGym

### Validate the environment

In [4]:
from stable_baselines3.common.env_checker import check_env

env = SimpleSimGym(max_budget=500, max_targets=3, num_classes=10, player_fov=60)

# If the environment don't follow the interface, an error will be thrown
check_env(env, warn=True)

Environment seed: 70


## Import Auto Saving of Best Model Callback

In [5]:
from utils import SaveOnBestTrainingRewardCallback

## Train The Model

In [6]:
from stable_baselines3 import PPO, SAC, TD3
from stable_baselines3.common.env_util import make_vec_env

# Environment Parameters
MAX_BUDGET = 400
MAX_TARGETS = 5
NUM_CLASSES = 10
PLAYER_FOV = 30
RENDER_MODE = "rgb_array"
ACTION_FORMAT = "continuous"

config = {
    "policy": 'MlpPolicy',
    "total_timesteps": 4_000_000,
    "logdir": "logs/",
    "savedir": "saved_models/",
}

# Create log dir
os.makedirs(config["logdir"], exist_ok=True)

# Create save dir
os.makedirs(config["savedir"], exist_ok=True)

# Show Tensorboard Logs
Visualise the live logs on tensorboard as we train

In [7]:
# Open Tensorboard Logging
%tensorboard --logdir logs/ --reload_multifile True --reload_interval 30

Reusing TensorBoard on port 6006 (pid 86057), started 0:28:11 ago. (Use '!kill 86057' to kill it.)

### Train SAC

In [8]:
# Instantiate and wrap the env
env_sac = make_vec_env(SimpleSimGym, 
                   n_envs=1, 
                   monitor_dir=config["logdir"]+"sac", 
                   env_kwargs=dict(
                       max_budget=MAX_BUDGET, 
                       max_targets=MAX_TARGETS, 
                       num_classes=NUM_CLASSES, 
                       player_fov=PLAYER_FOV, 
                       render_mode=RENDER_MODE, 
                       action_format=ACTION_FORMAT))

# Setup callbacks
auto_save_callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=config["logdir"], save_dir=config["savedir"], model_name="sac", verbose=0)

# Create the agent
model_sac = SAC(config["policy"], env_sac, tensorboard_log=config["logdir"], verbose=0)

# Train in tranches
times_trained = 0
num_tranches = 20
tranches = num_tranches*(1+times_trained)

for i in range((num_tranches * times_trained)+1, tranches+1):
    print(f"^ Tranch {i}/{tranches}")
    if i == 1:
        model_sac.learn(config["total_timesteps"]//tranches, tb_log_name="SAC", callback=auto_save_callback, progress_bar=True, reset_num_timesteps=True)
    else:
        model_sac.learn(config["total_timesteps"]//tranches, tb_log_name="SAC", callback=auto_save_callback, progress_bar=True, reset_num_timesteps=False)
    model_sac.save(f"{config['savedir']}/{config['policy']}_SAC_step{i * (config['total_timesteps']//tranches)}")

Environment seed: 975


Output()

^ Tranch 1/20


Saving new best model at 928 timesteps (saved to saved_models/best_sac) (reward=137.78222575)

Saving new best model at 1899 timesteps (saved to saved_models/best_sac) (reward=152.0928941764706)

...

Output()

^ Tranch 6/20


Saving new best model at 1125987 timesteps (saved to saved_models/best_sac) (reward=794.9330362799999)

Output()

^ Tranch 7/20


### Continue Training or Run Dupliacte Experiments?

This cell can be used to either continue training on an existing model (use `reset_num_timesteps=False`) or to run additional duplicate experiments training from scratch to test training consistency

In [ ]:
# model_sac.learn(config["total_timesteps"], tb_log_name="SAC", callback=auto_save_callback, progress_bar=True, reset_num_timesteps=False)
# model_sac.save(f"{config['savedir']}/{config['policy']}_SAC_pt2")

### Load Model?

In [ ]:
# # Load the best model
# model_sac = SAC.load(f"{config['savedir']}/MlpPolicy_SAC_step4000000")
best_sac = SAC.load(f"{config['savedir']}/best_sac")

### Check Performance

Check if the policy can consistently succeed in the environment over multilpe episodes.

In [ ]:
# Instantiate the eval env
eval_env = make_vec_env(SimpleSimGym, 
                   n_envs=1, 
                   # monitor_dir=config["logdir"], 
                   env_kwargs=dict(
                       max_budget=MAX_BUDGET, 
                       max_targets=MAX_TARGETS, 
                       num_classes=NUM_CLASSES, 
                       player_fov=PLAYER_FOV, 
                       render_mode=RENDER_MODE, 
                       action_format=ACTION_FORMAT
                   )
                  )

from stable_baselines3.common.evaluation import evaluate_policy

# Check performance of best vs last model
models = {"last_sac": model_sac, "best_sac": best_sac}#, "last_ppo": model_ppo, "best_ppo": best_ppo}
# models = {"best_sac": best_sac, "best_ppo": best_ppo}

for key in models.keys():
    # Reset the eval env
    eval_env.reset()
    # Test average reward over multiple episodes
    mean_reward, std_reward = evaluate_policy(models[key], eval_env, n_eval_episodes=50)
    print(f"MODEL TYPE: {key}")
    print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}\n")

### Visualize Trained Agent with Video


In [ ]:
from utils import record_video, show_videos

record_video(eval_env, model_sac, video_length=500*3, prefix="jupyter-sac-last-simplesim")
show_videos("videos", prefix="jupyter-sac-last")

record_video(eval_env, best_sac, video_length=500*3, prefix="jupyter-sac-best-simplesim")
show_videos("videos", prefix="jupyter-sac-best")